In [ ]:
import os
import numpy as np
import shutil
import cv2


In [ ]:
# Diretório para os pontos coletados
DATA_DIR = os.path.join('Dados_extraidos') 
VOLUNT_DIR = os.path.join('Dados_voluntarios')
 
# Sinais
sinais = np.array(['Ola','Tudo bem','Obrigado','Entendo','Calma'])

# Quantidade de vídeos por sinal
num_videos_sinal = 10

# Quantidade total de videos
num_videos_total = 1000

#Quantidade de coordenadas por frame
num_coordenadas = 1662

# Quantidade de voluntarios
num_voluntarios = 10

# Quantidade de frames por vídeo
num_frames = 30

In [ ]:
# Monta pasta Dados extraidos completa
for sinal in sinais: 
    try: 
        os.makedirs(os.path.join(DATA_DIR, sinal))
    except:
        pass

In [ ]:
# Renomeia todos os videos dos voluntarios
for sinal in sinais:

    video_num_total = 0
    
    for voluntario in range(num_voluntarios):

        for video_num in range(num_videos_sinal):

            nome_old = os.path.join(VOLUNT_DIR, str(voluntario), sinal, str(video_num))
            nome_new = os.path.join(VOLUNT_DIR, str(voluntario), sinal, str(video_num_total))
            os.rename(nome_old, nome_new)
            video_num_total +=1

In [ ]:
# Une todos os videos na pasta Dados_extraidos
for sinal in sinais:

    video_num_total = 0
    
    for voluntario in range(num_voluntarios):

        for video_num in range(num_videos_sinal):

            origem = os.path.join(VOLUNT_DIR, str(voluntario), sinal, str(video_num_total))
            destino = os.path.join(DATA_DIR, sinal)
            shutil.move(origem, destino)
            video_num_total +=1

In [ ]:
# Data Augmentation - Espelhamento horizontal dos vídeos 

for sinal in sinais:

    lista_videos = os.listdir(os.path.join(DATA_DIR, sinal))
    quant_videos = len(lista_videos)

    for video_num in np.array(lista_videos).astype(int):

        novo_video_num = video_num + quant_videos
        os.makedirs(os.path.join(DATA_DIR, sinal, str(novo_video_num)))

        for frame_num in range(num_frames):

            frame_npy = os.path.join(DATA_DIR, sinal, str(video_num), "{}.npy".format(frame_num))
            frame = np.load(frame_npy)

            for indice in range(num_coordenadas):

                if indice < 132:
                    if not (indice%4):
                        frame[indice] = 1 - frame[indice]
                else:
                    if not (indice%3):
                        frame[indice] = 1 - frame[indice]

            nome_arquivo = os.path.join(DATA_DIR, sinal, str(novo_video_num), str(frame_num))
            np.save(nome_arquivo, frame)                

In [ ]:
# Data Augmentation - Espelhamento horizontal dos vídeos 

for sinal in sinais:

    lista_videos = os.listdir(os.path.join(DATA_DIR, sinal))
    quant_videos = len(lista_videos)

    for video_num in np.array(lista_videos).astype(int):

        novo_video_num = video_num + quant_videos
        os.makedirs(os.path.join(DATA_DIR, sinal, str(novo_video_num)))

        for frame_num in range(num_frames):

            frame_npy = os.path.join(DATA_DIR, sinal, str(video_num), "{}.npy".format(frame_num))
            frame = np.load(frame_npy)

            frame_bracos = frame[52:92] 
            frame_maos = frame[1536:]  

            frame_bracos_espelhado = espelha_bracos(frame_bracos)
            frame_maos_espelhado = espelha_maos(frame_maos)
                          
            # Novo frame espelhado
            frame_espelhado = np.concatenate([frame[:52], 
                                            frame_bracos_espelhado, 
                                            frame[92:1536], 
                                            frame_maos_espelhado])

            nome_arquivo = os.path.join(DATA_DIR, sinal, str(novo_video_num), str(frame_num))
            np.save(nome_arquivo, frame_espelhado)                  

In [ ]:
def espelha_bracos(frame_bracos):

    # Espelhamento dos braços
    for indice in range(len(frame_bracos)):
        # Coordenadas x do corpo
        if not (indice%4):
                frame_bracos[indice] = 1 - frame_bracos[indice]

    # Inversão dos braços
    pontos = []
    frame_bracos_espelhado = []
    for indice in range(len(frame_bracos)):
        pontos.append(frame_bracos[indice])
        if len(pontos) == 8:
            ponto_esquerdo = pontos[:4]
            ponto_direito = pontos[4:]
            frame_bracos_espelhado.append(ponto_direito)
            frame_bracos_espelhado.append(ponto_esquerdo)
            pontos.clear()

    return np.array(frame_bracos_espelhado).flatten()

In [ ]:
def espelha_maos(frame_maos):

    # Espelhamento das mãos
    for indice in range(len(frame_maos)):
        # Coordenadas x das mãos
        if not (indice%3):
            frame_maos[indice] = 1 - frame_maos[indice]

    # Inversão das mãos
    frame_maos_espelhado = []
    frame_mao_esquerda = frame_maos[:63]
    frame_mao_direita = frame_maos[63:]
    frame_maos_espelhado.append(frame_mao_direita)
    frame_maos_espelhado.append(frame_mao_esquerda)

    return np.array(frame_maos_espelhado).flatten()

In [ ]:
# Prova real

frame_npy = os.path.join(DATA_DIR, 'Ola', '0', "0.npy")
frame = np.load(frame_npy)

for indice in range(num_coordenadas):
    if indice < 132:
        if not (indice%4):
            frame[indice] = 1 - frame[indice]
    else:
        if not (indice%3):
            frame[indice] = 1 - frame[indice]

frame_npy = os.path.join(DATA_DIR, 'Ola', '0', "0.npy")
frame_antes = np.load(frame_npy)

frame = frame + frame_antes

frame_corpo = frame[:132]
print(np.unique(frame_corpo[::4]))

frame_resto = frame[132:]
print(np.unique(frame_resto[::3]))

In [19]:
# Data Augmentation - Adicionando ruído gaussiano pra cada coordenada

for sinal in sinais:

    lista_videos = os.listdir(os.path.join('Dados_extraidos_1000', sinal))
    quant_videos = len(os.listdir(os.path.join('Dados_extraidos', sinal)))

    for video_num in np.array(lista_videos).astype(int):

        novo_video_num = video_num + quant_videos
        os.makedirs(os.path.join('Dados_extraidos', sinal, str(novo_video_num)))

        for frame_num in range(num_frames):

            frame_npy = os.path.join('Dados_extraidos_1000', sinal, str(video_num), "{}.npy".format(frame_num))
            frame = np.load(frame_npy)
            ruido = gera_ruido_gaussiano(-0.1, 0.1, 0, 0.05, frame.shape[0])
            frame = frame + ruido
            nome_arquivo = os.path.join('Dados_extraidos', sinal, str(novo_video_num), str(frame_num))
            np.save(nome_arquivo, frame)                

In [18]:
# Data Augmentation - Adicionando ruído gaussiano pra cada coordenada

for sinal in sinais:

    lista_videos = os.listdir(os.path.join(DATA_DIR, sinal))
    quant_videos = len(os.listdir(os.path.join(DATA_DIR, sinal)))

    for video_num in np.array(lista_videos).astype(int):

        novo_video_num = video_num + quant_videos
        os.makedirs(os.path.join(DATA_DIR, sinal, str(novo_video_num)))

        for frame_num in range(num_frames):

            frame_npy = os.path.join(DATA_DIR, sinal, str(video_num), "{}.npy".format(frame_num))
            frame = np.load(frame_npy)
            ruido = gera_ruido_gaussiano(-0.1, 0.1, 0, 0.05, frame.shape[0])
            frame = frame + ruido
            nome_arquivo = os.path.join(DATA_DIR, sinal, str(novo_video_num), str(frame_num))
            np.save(nome_arquivo, frame)                

In [ ]:
import scipy.stats as scs

def gera_ruido_gaussiano(lim_inferior, lim_superior, centro, desvio, qtd_amostras):
    return scs.truncnorm.rvs(a=(lim_inferior-centro)/desvio, 
                                b=(lim_superior-centro)/desvio, 
                                loc=centro,scale=desvio,
                                size=qtd_amostras)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(2)
ax[0].hist(ruido)
plt.show()